<font color='green'><b>Полезные (и просто интересные) материалы:</b></font> \
Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множествореализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html 

<font color='green'>Пример BERT с GPU:
```python
%%time
from tqdm import notebook
batch_size = 2 # для примера возьмем такой батч, где будет всего две строки датасета
embeddings = [] 
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids[batch_size*i:batch_size*(i+1)]).cuda() # закидываем тензор на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()
        
        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings
        
features = np.concatenate(embeddings) 
```
Можно сделать предварительную проверку на наличие GPU.\
Например, так: ```device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")```\
Тогда вместо .cuda() нужно писать .to(device)

Если понравилась работа с текстами, можно посмотреть очень интересный (но очень-очень сложный) курс лекций: https://github.com/yandexdataschool/nlp_course .
</font>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords as stopwords_nltk
import re
from pymystem3 import Mystem

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from nltk.stem import WordNetLemmatizer

from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from random import randint


In [2]:
nltk.download('stopwords')

NameError: name 'nltk' is not defined

In [ ]:
RANDOM_STATE = 12345

In [ ]:
stopwords = set(stopwords_nltk.words('english'))

In [4]:
df = pd.read_csv('datasets/toxic_comments.csv')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
# Взвесим классы
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Классы не сбалансированы.

In [6]:
# Очистим тексты от посторонних символов
df['text'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x).split())
df['text'] = df['text'].apply(lambda x: ' '.join(x))

display(df.head())

,text,toxic
0,Explanation Why the edits made under my userna...,0
1,D aww He matches this background colour I m se...,0
2,Hey man I m really not trying to edit war It s...,0
3,More I can t make any real suggestions on impr...,0
4,You sir are my hero Any chance you remember wh...,0


In [7]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = text.split()
    return ' '.join([lemmatizer.lemmatize(w) for w in words])

In [8]:
# Лемматизируем тексты корпуса
df['lemm_text'] = df['text'].apply(lemmatize_text)
df.head()

,text,toxic,lemm_text
0,Explanation Why the edits made under my userna...,0,Explanation Why the edits made under my userna...
1,D aww He matches this background colour I m se...,0,D aww He match this background colour I m seem...
2,Hey man I m really not trying to edit war It s...,0,Hey man I m really not trying to edit war It s...
3,More I can t make any real suggestions on impr...,0,More I can t make any real suggestion on impro...
4,You sir are my hero Any chance you remember wh...,0,You sir are my hero Any chance you remember wh...


In [10]:
train_df, other_df = train_test_split(df, test_size=0.25, random_state=RANDOM_STATE)
valid_df, test_df = train_test_split(other_df, test_size=0.5, random_state=RANDOM_STATE)

features_train = train_df.drop(['toxic'], axis=1)
features_valid = valid_df.drop(['toxic'], axis=1)
features_test = test_df.drop(['toxic'], axis=1)

display(features_train.head())
display(features_valid.head())
display(features_test.head())

,text,lemm_text
111565,Ath Cliath section We have a bit of a problem ...,Ath Cliath section We have a bit of a problem ...
8575,Sure thing By the way I have a new userbox tha...,Sure thing By the way I have a new userbox tha...
153402,We are in the same boat as Britannica which is...,We are in the same boat a Britannica which is ...
65019,I had a look through the section on theology a...,I had a look through the section on theology a...
155787,Warren Commission Exhibit E the Chin It is tim...,Warren Commission Exhibit E the Chin It is tim...


,text,lemm_text
82956,Please do not vandalize pages as you did with ...,Please do not vandalize page a you did with th...
56103,Once again I ve translated via Google and had ...,Once again I ve translated via Google and had ...
105523,I just wanted to say your article sucks,I just wanted to say your article suck
39712,Trouted You have been trouted for Fuck you mot...,Trouted You have been trouted for Fuck you mot...
3621,Thanks I d seen the Lu s I pagemove request at...,Thanks I d seen the Lu s I pagemove request at...


,text,lemm_text
110631,You really ought to read WP FAQ Business it s ...,You really ought to read WP FAQ Business it s ...
127210,Retroactive thinking What are having retroacti...,Retroactive thinking What are having retroacti...
7437,vindinctive Apparently you have not looked at ...,vindinctive Apparently you have not looked at ...
123241,sabotaging this article on the eternity clause...,sabotaging this article on the eternity clause...
157381,Excuse me who the fuck do you think you are de...,Excuse me who the fuck do you think you are de...


In [11]:
target_train = train_df['toxic']
target_valid = valid_df['toxic']
target_test = test_df['toxic']

print(target_train.head())
print(target_valid.head())
print(target_test.head())

111565    0
8575      0
153402    0
65019     0
155787    0
Name: toxic, dtype: int64
82956     0
56103     0
105523    1
39712     1
3621      0
Name: toxic, dtype: int64
110631    0
127210    0
7437      0
123241    0
157381    1
Name: toxic, dtype: int64


In [12]:
# Добавим в данные признак tf idf
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

tf_idf_train = count_tf_idf.fit_transform(features_train['lemm_text'])
tf_idf_valid = count_tf_idf.transform(features_valid['lemm_text'])
tf_idf_test = count_tf_idf.transform(features_test['lemm_text'])

print(tf_idf_train)
print(tf_idf_valid)
print(tf_idf_test)

  (0, 70552)	0.17164141019015153
  (0, 35705)	0.17164141019015153
  (0, 26110)	0.09628922580242742
  (0, 131328)	0.1460118306101786
  (0, 88154)	0.06587833467946713
  (0, 9939)	0.15853465862316918
  (0, 70271)	0.1604930272468847
  (0, 107255)	0.05210639418651967
  (0, 12863)	0.06254639177669576
  (0, 23804)	0.06293492318825762
  (0, 5935)	0.06524593318376669
  (0, 128265)	0.07643185311733816
  (0, 35714)	0.1315417488194532
  (0, 131327)	0.13609695580142506
  (0, 95184)	0.0990454097706476
  (0, 101992)	0.10978840819340827
  (0, 12779)	0.0651035605844043
  (0, 42318)	0.138704909005662
  (0, 134223)	0.07301907974451072
  (0, 36221)	0.17164141019015153
  (0, 35820)	0.17164141019015153
  (0, 18079)	0.20781862445117458
  (0, 66680)	0.04756814474347022
  (0, 135969)	0.06245261657149667
  (0, 68674)	0.075766842401091
  :	:
  (119677, 41881)	0.2853392513138432
  (119677, 132078)	0.24060471490351415
  (119677, 33349)	0.12203640337051935
  (119677, 90101)	0.1118481938132253
  (119677, 63839)	0.15

<div class="alert alert-block alert-info">
Векторизатор обучаем только после разбиения выборки на части. При этом он должен быть обучен только на тренировочной части данных.
</div>

1. Очистили тексты выборки от посторонних символов, провели лемматизацию.
2. Преобразовали тексты в вектора с помощью модели TF-IDF (оценки важности слов).
2. Классы в выборке не сбалансированы. При обучении моделей сбалансируем классы.

## Обучение

### Логистическая регрессия

In [13]:
model_lg = LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced', solver='lbfgs', max_iter=1000)
model_lg.fit(tf_idf_train, target_train)

pred_valid = model_lg.predict(tf_idf_valid)
f1_valid = f1_score(target_valid, pred_valid)

print("F1 наилучшей модели 'Логистическая регрессия' на валидационной выборке:", f1_valid)


F1 наилучшей модели 'Логистическая регрессия' на валидационной выборке: 0.7477516059957172


### Градиентный бустинг

In [14]:
cat = CatBoostClassifier(random_state=RANDOM_STATE, learning_rate=0.1, auto_class_weights='Balanced', iterations=50)
cat.fit(tf_idf_train, target_train)

pred_valid = cat.predict(tf_idf_valid)
f1_valid = f1_score(target_valid, pred_valid)

print("F1 наилучшей модели 'Градиентный бустинг' на валидационной выборке", f1_valid)


0:	learn: 0.6520324	total: 3.07s	remaining: 2m 30s
1:	learn: 0.6215697	total: 5.33s	remaining: 2m 7s
2:	learn: 0.5974717	total: 7.57s	remaining: 1m 58s
3:	learn: 0.5849294	total: 9.84s	remaining: 1m 53s
4:	learn: 0.5704476	total: 12.2s	remaining: 1m 49s
5:	learn: 0.5587096	total: 14.4s	remaining: 1m 45s
6:	learn: 0.5497374	total: 16.6s	remaining: 1m 42s
7:	learn: 0.5422540	total: 18.9s	remaining: 1m 39s
8:	learn: 0.5331295	total: 21.1s	remaining: 1m 35s
9:	learn: 0.5260133	total: 23.3s	remaining: 1m 33s
10:	learn: 0.5212618	total: 25.5s	remaining: 1m 30s
11:	learn: 0.5136164	total: 27.7s	remaining: 1m 27s
12:	learn: 0.5068651	total: 29.9s	remaining: 1m 24s
13:	learn: 0.5027035	total: 32.2s	remaining: 1m 22s
14:	learn: 0.4986257	total: 34.3s	remaining: 1m 19s
15:	learn: 0.4946007	total: 36.4s	remaining: 1m 17s
16:	learn: 0.4903612	total: 38.6s	remaining: 1m 14s
17:	learn: 0.4865575	total: 40.5s	remaining: 1m 12s
18:	learn: 0.4807599	total: 42.8s	remaining: 1m 9s
19:	learn: 0.4779761	tot

In [15]:
print(cat.get_all_params())

{'nan_mode': 'Min', 'eval_metric': 'Logloss', 'iterations': 50, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 3, 'random_strength': 1, 'rsm': 1, 'boost_from_average': False, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'subsample': 0.800000011920929, 'use_best_model': False, 'class_names': [0, 1], 'random_seed': 12345, 'depth': 6, 'posterior_sampling': False, 'border_count': 254, 'class_weights': [1, 8.859779357910156], 'classes_count': 0, 'auto_class_weights': 'Balanced', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1, 'model_shrink_rate': 0, 'min_data_in_leaf': 1, 'loss_function': 'Logloss', 'learning_rate': 0.100000001490116

### Решающее дерево

In [16]:
best_model_tree = None
best_f1_valid = 0
best_depth_tree = 0

for depth in range(1, 10):
    model_tree = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth, class_weight='balanced')
    model_tree.fit(tf_idf_train, target_train)
    
    pred_valid = model_tree.predict(tf_idf_valid)
    f1_valid = f1_score(target_valid, pred_valid)

    if f1_valid > best_f1_valid:
        best_model_tree = model_tree
        best_f1_valid = f1_valid
        best_depth_tree = depth
        
print("F1 наилучшей модели 'Дерево решений' на валидационной выборке:", best_f1_valid, "Глубина дерева:", best_depth_tree)


F1 наилучшей модели 'Дерево решений' на валидационной выборке: 0.5253893026404874 Глубина дерева: 8


### Случайный лес

In [17]:
model_forest = RandomForestClassifier()
params = {
    'max_depth': [randint(1, 100)],
    'n_estimators': [randint(1, 100)],
    'random_state': [RANDOM_STATE],
    'class_weight': ['balanced']
}

grid = GridSearchCV(model_forest, params, scoring='f1')
grid.fit(tf_idf_train, target_train)
print(grid.best_params_)

{'class_weight': 'balanced', 'max_depth': 74, 'n_estimators': 19, 'random_state': 12345}


In [18]:
def search_forest(depth, est):
    model_forest = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=RANDOM_STATE, class_weight='balanced')
    model_forest.fit(tf_idf_train, target_train)

    pred_valid = model_forest.predict(tf_idf_valid)
    f1_valid = f1_score(target_valid, pred_valid)
    
    print("F1 наилучшей модели 'Случайный лес' на валидационной выборке", f1_valid)
    

In [19]:
search_forest(48, 82)

F1 наилучшей модели 'Случайный лес' на валидационной выборке 0.46526036288538125


### Проверка на тестовой выборке

In [20]:
pred_test = model_lg.predict(tf_idf_test)
f1_test = f1_score(target_test, pred_test)

print("F1 наилучшей модели 'Логистическая регрессия' на тестовой выборке:", f1_test)

F1 наилучшей модели 'Логистическая регрессия' на тестовой выборке: 0.7544238239102288


## Выводы

Лучшей моделью оказалась Логистическая регрессия (значение метрики `f1` на тестовой выборке равно 0.7544).